## Exercise 1 ~ A full round trip? Ain’t nobody got time for that!

In [1]:
import socket
import hashlib
import binascii
import utils
# Get variable from parameters file
params = utils.get_parameters()

In [2]:
p = params.Q1_p
q = params.Q1_q
g = params.Q1_g
pk = params.Q1_pk
sk = params.Q1_sk

In [3]:
print p

60055542378743800870984650064300434749863701922850044406431317093041369825985827433749546393895540449652025734272324325766218491281694238130722173393183483885437811760557394111938168661132034686043294099496516186034464224769515668824134222379411213622533288466031472830765575070050471211910512309277394625511039608604466794511486442978890755447340853298788925954132849494246405952766545954686681263626373302433301707202080315167178326221865198357311916159655281789965111665708959524382410794912853864178879967461309707005917351897844330144906695159023225938171656971885521551778839866449272779226789329


In [4]:
print q

946369383149503913097499114667945567529167788584664587545246699802207806093


In [5]:
print g

22493437217519503842042074486862187182867236296232324847042667312311970684978888420271944247698412501149798133622445424915931079950752980864405692539831020246953893352433357520362036811397373869313049949339622279744263417333262727784136625059611007301281594242808226739999769715591223297216365080543802674283076625008089439983472611892820555332588466330669644978677276389400870722474404883111442691947196879407967650259989043439266848164547113521977979571296250750760221388613710711549516681789079332853324508231151989505119703100522561290674805773702857445297920694801675555969359295881697509797401703


In [6]:
print pk

14756448914390602869910211060015458444881608441611746640037742789207089044306375571534720691481804676330815806912646756382029857169292655819657909765587412374312476546703616935804401538679605879713514445214373592817294444138692615283642233174762126785417450490574765976150068579610840190825798712569306850505863845265195678018800439728480653487717038149046357211022379343160776731995216744010536204571885920381738544950320830710846338259627881671060097684057708632933637733249470867478433938592565039355267389976999560888386203097632782135850143539530764774290521841539013884252348919732554825131183394


In [7]:
print sk

842686782033015526616049802372041293120901487222997733015409592779893766009


In [8]:
def connect_server(server_name, port, message):
    server = (server_name, int(port)) #calling int is required when using Sage
    s = socket.create_connection(server)
    s.send(bytes(message + "\n"))
    response=""
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(1024)
        data = data.decode("ASCII")
        if len(data) == 0:
            break
        if data[-1] == '\n': 
            response += data[:-1]  
            break
        response += data
    s.close()
    return response


def oracle_query(sciper):
    server_name = "lasecpc25.epfl.ch"
    port = "5559"
    message = sciper 
    response = connect_server(server_name, port, message)
    #print(response)
    return response

# Returns a binary oracle_query
def bin_query():
    return bin(int(oracle_query("247565"), 16))[2:].zfill(16752)

In [9]:
print oracle_query("247565")

d8e7082b5ef8f71eea83e8e6d8976168b372e3f9c844d77b153c50d3d17ffbb26aaedf76cac81513eee323d48c063c4f0aeee83514a7df9bccbd75287b56ead169583e5789db2e4792ef35ee8e8c3b9bae5d85a3324d1ca05f8ae79681b1de05010ef144ab1dc51834ee7b81823c99d0965ec6eb39c67cba7a232e85a7b8a49f31ba677e3868f485efcf61b6f4624c32c8e1106494204fdedd99911fca07c6f541e0b2c53733235d5538b1f98a02244e6ea7a90b11a7a08c4acaaffad8afb542e1cfec3e86cecdcf8326ce8e27bdf709d49863516b1da68edc1ad6ecbd365e14bae9f6768a4216bd0332fb383e37ed34d56e6cf891b5e051c69795af1ed164e63ce229a0c96204a83bfb5a6ae6f6f18f58bedb21ef99af2cdb20f5f3b388d753c80e98e786f5d59fb69d43e25e5b5725526bec80021da652a820155814c47e3e1cd6467fb3179c10db591dc8d9115e30c9c206f771c84bdea6d3cf038eb1944c8b25cf12b0d29f105359334550a71de166744aecfa63e10ff8d62584ff12c043f78a5cd8255cea94f8b2018581152ba2509566b435fad6fb0636b3ad900ce4e4c6a5240b1be0a12f01bd1c0297a89cb40b5465cae940a35fbbf462e09dd0ceb17709970af64b7f2f23c2cd8bffa804d6a7c44feba0cef2de6dea1072bd5441eee556c7967014e4b28e415194fbff881196294710

- iterate over the blocks (2000 mixed with 256) of the query and find the value $a$ for which $a^q=1\textrm{ mod }p$ and $a!=1\textrm{ mod }p$ => $a$ is $X_0$.
- Once we have $X_0$, we can find $pk^{x_0}$ with $X_0^{sk}$
- Then, we try to decrypt the blocks with the St values we compute
- We find the block with *password*

In [56]:
def encode(a, n=256):
    x = bin(a)[2:].zfill(n)
    return x[-n:]

def kdf(st):
    hex_value = hex(int(st, 2))[2:]
    if hex_value[-1] == 'L':
        hex_value = hex_value[:-1]
    hex_value = hex_value.zfill(64)
    k_to_hash = binascii.unhexlify("00" + hex_value)
    s_to_hash = binascii.unhexlify("ff" + hex_value)
    k = bin(int(hashlib.sha256(k_to_hash).hexdigest(), 16))[2:].zfill(256)
    s = bin(int(hashlib.sha256(s_to_hash).hexdigest(), 16))[2:].zfill(256)
    return s, k

def xor(a, b):
    c = ''
    for i in range(len(a)):
        if a[i] == b[i]:
            c += '0'
        else:
            c += '1'
    return c

In [60]:
result = ''

while result == '':
    query = bin_query()
    integers = []
    blocks = []
    check = 0
    while len(integers) < 3:
        if check + 2000 > len(query):
            break
        X = int(query[check:check+2000], 2)
        if power_mod(X, q, p) == 1:
            integers.append(X)
            check += 2000
        else:
            blocks.append(query[check:check+256])
            check += 256
    for i in integers:
        St = encode(power_mod(i, sk, p))
        for _ in range(len(blocks)):
            St, ki = kdf(St)
            for block in blocks:
                m = xor(block, ki)
                n = int(m, 2)
                try:
                    s = binascii.unhexlify('%x' % n)
                    if s.find("password") != -1:
                        result = s
                except:
                    ''

print result

.com&password=qmK-GnH-6WM-bgo&ti
